<a href="https://colab.research.google.com/github/innocentmatutu/Machine-learning/blob/main/Tsunami_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
from os import X_OK
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

df=pd.read_csv('/content/archive (7).zip')

#Encoding of categorical columns(just for formality since data does not contain it)
for col in df.select_dtypes(include=['object']).columns:
   encoder = LabelEncoder()
   df[col] = encoder.fit_transform(df[col])

#Filling missing values with the mean along each column
def impute_with_extension(X_train,X_valid):
  cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
  X_train_plus = X_train.copy()
  X_valid_plus = X_valid.copy()

  for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

  imputer = SimpleImputer(strategy='mean')
  imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train_plus),columns = X_train_plus.columns)
  imputed_X_valid = pd.DataFrame(imputer.transform(X_valid_plus),columns = X_valid_plus.columns)

  return imputed_X_train, imputed_X_valid

#Feature and target selection
#features = ['longitude','Year','nst','dmin','cdi','gap','latitude']
features = ['longitude','Year','latitude','dmin']
X = df[features]
y = df['tsunami']
df.drop(['tsunami'],axis=1,inplace=True)

#Training,testing and spliting of data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1)

#Calling of the imputed data columns
imputed_X_train, imputed_X_valid = impute_with_extension(X_train,X_valid)

#Model selection,training and prediction
model = xgb.XGBClassifier(n_estimators=500,learning_rate=0.01,max_depth=6,n_jobs=1)
model.fit(imputed_X_train,y_train)
prediction = model.predict(imputed_X_valid)

#metrics
print('Accuracy score:', accuracy_score(y_valid,prediction))
print('Precision score:', precision_score(y_valid,prediction))
print('Recall score:',recall_score(y_valid,prediction))
print('F1 score:', f1_score(y_valid,prediction))



Accuracy score: 0.9235668789808917
Precision score: 0.8870967741935484
Recall score: 0.9166666666666666
F1 score: 0.9016393442622951
